In [1]:
import pandas as pd
import numpy as np
import os
from ete3 import NCBITaxa, PhyloTree
ncbi = NCBITaxa()
from collections import Counter
import gseapy as gp
from gseapy.plot import barplot, dotplot

In [2]:
table_path='C:\\Users\\huangyan8\\Desktop\\work\\2024-09-03 TE_Evolution_Draft\\Tables\\'
fig_path='C:\\Users\\huangyan8\\Desktop\\work\\2024-09-03 TE_Evolution_Draft\\Figures\\'
samples=pd.read_excel(table_path+'Table S1\\Table S1.TE_Evo_352_Species.xlsx')
samples.shape

(352, 7)

### KEGG Enrichment of TE-associated Genes

In [3]:
# prepare KEGG information
import re
p=re.compile("K\d{5}")
path='D:\\18.TE_Evolution\\03.Protein_Annotation\\KEGG_Result\\KEGG_Result\\KEGG_Result\\'
ko_info={}
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+'.kegg.path' in os.listdir(path):
        KO=pd.read_csv(path+specie_id+'.blast.kegg.xls',sep='\t')
        KO=KO.drop_duplicates('Subject_annotation').reset_index(drop=True)
        for j in range(KO.shape[0]):
            values=KO.loc[j,'Subject_annotation'].split("; ")
            for v in values:
                if p.findall(v)!=[]:
                    ko=p.findall(v)[0]
                    anno=v[7:]
                    ko_info[ko]=anno

0
100
200
300


In [5]:
kegg_path='D:\\18.TE_Evolution\\03.Protein_Annotation\\KEGG.ko\\'
gene_sets={}
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+".ko.csv" in os.listdir(kegg_path):
        df=pd.read_csv(kegg_path+specie_id+".ko.csv")
        df=df.dropna().reset_index(drop=True)
        for j in range(df.shape[0]):
            gene=df.loc[j,'Gene']
            kos=df.loc[j,'KEGG_Terms']
            if "\t" in kos:
                kos=kos.split("\t")
            elif " " in kos:
                kos=kos.split(" ")
            elif " | " in kos:
                kos=kos.split(" | ")
            else:
                kos=[kos]
            for ko in kos:
                if ko in gene_sets:
                    gene_sets[ko].append(gene)
                else:
                    gene_sets[ko]=[gene]

0
100
200
300


In [6]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part2.TE-associated_Genetic_Functions\\'
TEpep_genes_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
TE_dupgene_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Overall\\'
gene_list=[]
count=0
for  i in range(samples.shape[0]):
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+".TEpep_to_Gene.txt" in os.listdir(TEpep_genes_path):
        gene_list+= [gene.strip() for gene in open(TEpep_genes_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]
    if specie_id+".TE_DupGenes.txt" in os.listdir(TE_dupgene_path):
        gene_list+=[gene.strip() for gene in open(TE_dupgene_path+specie_id+".TE_DupGenes.txt",'r').readlines()]
gene_list=list(set(gene_list))
count=len(gene_list)
print('TE-associated Genes',count)
if len(gene_list)>10:
    try:
        enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
            outdir=save_path,#输出目录
            top_term=20,
            cutoff=0.01#pvalue阈值
                                             )
        result=enr.results
        result=result.sort_values("Adjusted P-value")
        result["Title"]=result['Term'].apply(lambda x:ko_info[x] if x in ko_info else '')
        print(family,result.shape)
        result.to_excel(save_path+"TE-associated_Genes_KEGG_Enrichment.xlsx",index=False)
    except:
        pass
else:
    print('Error: No TE-associated genes!')

TE-associated Genes 5871053


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


In [7]:
result[result["Adjusted P-value"]<0.01].shape

(1853, 9)

### KEGG Enrichment of Transpoase Domesticated Genes

In [8]:
TEpep_genes_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
gene_sets={}
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    bg_file=specie_id+".TEpep_to_Gene.txt"
    bg_genes={}
    if bg_file in os.listdir(TEpep_genes_path):
        for gene in [gene.strip() for gene in open(TEpep_genes_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]:
            bg_genes[gene]=1
    if specie_id+".ko.csv" in os.listdir(kegg_path):
        df=pd.read_csv(kegg_path+specie_id+".ko.csv")
        df=df.dropna().reset_index(drop=True)
        for j in range(df.shape[0]):
            gene=df.loc[j,'Gene']
            if gene in bg_genes:
                kos=df.loc[j,'KEGG_Terms']
                if "\t" in kos:
                    kos=kos.split("\t")
                elif " " in kos:
                    kos=kos.split(" ")
                elif " | " in kos:
                    kos=kos.split(" | ")
                else:
                    kos=[kos]
                for ko in kos:
                    if ko in gene_sets:
                        gene_sets[ko].append(gene)
                    else:
                        gene_sets[ko]=[gene]

0
100
200
300


In [9]:
gene_list_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Super_Family\\'
families=list(set([f.split(".TEpep")[0][:-5] for f in os.listdir(gene_list_path)]))
families

['LTR_Roo',
 'Crypton',
 'Mariner',
 'P_element',
 'CACTA',
 'hAT',
 'LINE',
 'mariner_ant1',
 'piggybac',
 'Gypsy',
 'Copia',
 'ISb',
 'Helitron',
 'Mutator',
 'DDE_1',
 'ISC1316']

In [ ]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part1.Family_Enrichment\\'
for family in families:
    gene_list=[]
    for  i in range(samples.shape[0]):
        specie_id=samples.loc[i,'Specie_ID']
        g_file=family+"_"+specie_id+'.TEpep_to_Gene.txt'#".TE_DupGenes.txt"
        if g_file in os.listdir(gene_list_path):
            genes=[gene.strip() for gene in open(gene_list_path+g_file,'r').readlines()]
            gene_list+=genes
    print(family,len(gene_list))
    if len(gene_list)>10:
        try:
            enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
                            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
                            outdir=save_path,#输出目录
                            top_term=20,
                            cutoff=0.01#pvalue阈值
                                             )
            result=enr.results
            result=result.sort_values("Adjusted P-value")
            result["Title"]=result['Term'].apply(lambda x:ko_info[x] if x in ko_info else '')
            print(family,result.shape)
            result.to_excel(save_path+family+"_KEGG_TEpep_Dom_Enrichment.xlsx",index=False)
        except:
            pass
    else:
        print(family)

LTR_Roo 4001


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


LTR_Roo (1017, 9)
Crypton 31
Crypton (19, 9)
Mariner 744


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Mariner (165, 9)
P_element 2
P_element
CACTA 54024


...

Gypsy (9896, 9)
Copia 354131


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Copia (13412, 9)
ISb 2
ISb
Helitron 16128


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Helitron (2316, 9)
Mutator 75681


### Enrichment of TE induced Dup Genes

In [ ]:
bg_genes={}
gene_sets={}
background_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Overall\\'
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    bg_file=specie_id+'.TE_DupGenes.txt'#".TE_DupGenes.txt"
    #rm_file=specie_id+".TEpep_to_Gene.txt"
    if bg_file in os.listdir(background_path):
        genes=[gene.strip() for gene in open(background_path+bg_file,'r').readlines()]
        for gene in genes:
            bg_genes[gene]=1
        if specie_id+".ko.csv" in os.listdir(kegg_path):
            df=pd.read_csv(kegg_path+specie_id+".ko.csv")
            df=df.dropna().reset_index(drop=True)
            for j in range(df.shape[0]):
                gene=df.loc[j,'Gene']
                if gene in bg_genes:
                    kos=df.loc[j,'KEGG_Terms']
                    if "\t" in kos:
                        kos=kos.split("\t")
                    elif " " in kos:
                        kos=kos.split(" ")
                    elif " | " in kos:
                        kos=kos.split(" | ")
                    else:
                        kos=[kos]
                    for ko in kos:
                        if ko in gene_sets:
                            gene_sets[ko].append(gene)
                        else:
                            gene_sets[ko]=[gene]

In [ ]:
gene_list_path='D:\\18.TE_Evolution\\07.DupGene_Finder\\TE_Dup_Genes(2K)\\Super_Family\\'
families=list(set([f.split(".TE")[0][:-5] for f in os.listdir(gene_list_path)]))
families

In [ ]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part1.Family_Enrichment\\'
tepep_path='D:\\18.TE_Evolution\\05.TEpep\\Gene_TEpep_Genelist\\Overall\\'
for family in families:
    gene_list=[]
    count=0
    for  i in range(samples.shape[0]):
        specie_id=samples.loc[i,'Specie_ID']
        tepep_genes={}
        if specie_id+".TEpep_to_Gene.txt" in os.listdir(tepep_path):
            for gene in [gene.strip() for gene in open(tepep_path+specie_id+".TEpep_to_Gene.txt",'r').readlines()]:
                tepep_genes[gene]=1
        g_file=family+"_"+specie_id+".TE_DupGenes.txt"
        if g_file in os.listdir(gene_list_path):
            genes=[gene.strip() for gene in open(gene_list_path+g_file,'r').readlines()]
            for gene in genes:
                if gene not in tepep_genes:
                    gene_list.append(gene)
                else:
                    count+=1
    print(family,len(gene_list),'TEpep Genes',count)
    if len(gene_list)>10:
        try:
            enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
                            gene_sets=gene_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
                            outdir=save_path,#输出目录
                            top_term=20,
                            cutoff=0.01#pvalue阈值
                                             )
            result=enr.results
            result=result.sort_values("Adjusted P-value")
            result["Title"]=result['Term'].apply(lambda x:ko_info[x] if x in ko_info else '')
            print(family,result.shape)
            result.to_excel(save_path+family+"_KEGG_TE_DupGenes_Enrichment.xlsx",index=False)
        except:
            pass
    else:
        print(family)